In [1]:
! pip install --no-index --find-links=/kaggle/input/monai-whl monai
! pip install --no-index --find-links=/kaggle/input/torchio torchio
! pip install --no-index --find-links=/kaggle/input/addict addict

Looking in links: /kaggle/input/monai-whl
Processing /kaggle/input/monai-whl/monai-1.1.0-202212191849-py3-none-any.whl
Looking in links: /kaggle/input/torchio
Processing /kaggle/input/torchio/torchio-0.18.86-py2.py3-none-any.whl
Processing /kaggle/input/torchio/Deprecated-1.2.13-py2.py3-none-any.whl
Processing /kaggle/input/torchio/shellingham-1.5.0.post1-py2.py3-none-any.whl
Looking in links: /kaggle/input/addict
Processing /kaggle/input/addict/addict-2.4.0-py3-none-any.whl


In [2]:
import os, sys, platform
from os.path import join, dirname, basename, abspath
import time
import numpy as np
import pandas as pd
import pydicom
import SimpleITK as sitk
import cv2
import torch
import tensorflow
import matplotlib.pyplot as plt
import yaml
import h5py
import json
from glob import glob
# import addict
from addict import Dict
# import monai
import monai
# import torchio
import torchio as tio
# import custom classes
import importlib

repo = "/kaggle/input/rsna-breast-cancer-2023"
os.chdir(repo)

import submission
importlib.reload(submission)
from submission import Submission

In [3]:
def main(cfile):
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    cfgs = Dict(yaml.load(open(abspath(cfile), "r"), Loader=yaml.Loader))
    submit = Submission(cfgs)
    start = time.time()
    submit.Run()
    submit.ExportSubmissionCSV()
    end = time.time()
    print("\n")
    start_s = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(start))
    end_s = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(end))
    time_taken = end - start
    if time_taken > 3600.:
        divisor = 3600.
        suffix = "hr"
    else:
        divisor = 60.
        suffix = "min"
    print(f"Start Time: {start_s}, End Time: {end_s}, Total Time Taken: {(time_taken)/divisor:.3f} {suffix}")

In [4]:
save_path = "/kaggle/working/submission.csv"
test_md_path = "/kaggle/input/rsna-breast-cancer-detection/train.csv"
test_md = pd.read_csv(test_md_path)
pats = test_md["patient_id"].tolist()
lats = test_md["laterality"].tolist()
pat_ids = ["_".join(item) for item in zip(map(str, pats), lats)]
rand_cancer = np.random.rand(len(pat_ids))
result = pd.DataFrame({"prediction_id": pat_ids, "cancer": rand_cancer})

## Preprocessing Test Images

In [5]:
continue_test = True
try:
    subprocess.run(["python", "preprocessing.py", "-c", "/kaggle/input/rsna-breast-cancer-2023/config/test_config.yaml"])
    subprocess.run(["python", "splitdata.py", "-c", "/kaggle/input/rsna-breast-cancer-2023/config/test_config.yaml"])
except:
    continue_test = False
    result.to_csv(save_path, index=False)

## Load and run model

In [6]:
if continue_test:
    try:
        config_file = "/kaggle/input/rsna-breast-cancer-2023/config/test_config.yaml"
        main(config_file)
    except:
        result.to_csv(save_path, index=False)

In [7]:
os.chdir("/kaggle/working/")